In [254]:
%matplotlib notebook
import pandas as pd
import numpy as np
import dask.dataframe as dk

# Functions

In [255]:
def euclidian_distance(p1,p2):
    return np.sqrt((p1.latitude - p2.latitude)**2+(p1.longitude - p2.longitude)**2)

def closest_station(row):
    min_euc=10000
    neighbor=''
    for i,j,k in zip(latitude,longitude,s_id):
        comp=pd.DataFrame({'latitude':[i],'longitude':[j]})
        euc_d=euclidian_distance(row,comp)
        if euc_d[0]<min_euc and euc_d[0] >0:
            min_euc,neighbor=euc_d[0],k
    return neighbor

def fill_from_neighbor(col):
    neighbor=y_localisation.loc[y_localisation.t_id==col.name]

def pred(row):
    weekday = row.timestamp.weekday()
    hour=row.timestamp.hour
    mins=row.timestamp.minute
    for i in y_train.columns[1:]:
        row[i]=y_train.loc[(y_train['timestamp'].dt.weekday==weekday) & (y_train['timestamp'].dt.hour==hour) & (y_train['timestamp'].dt.minute==mins)][i].mode().values[0]
    return row


# Loading data

In [256]:
y_train=pd.read_csv('ytrain_NpxebDC.csv')
y_train['timestamp']=pd.to_datetime(y_train['timestamp'])
y_localisation=pd.read_csv('sup_files/info_static.csv')
ypred=pd.read_csv('yrandom.csv',usecols=[0])
ypred['timestamp']=pd.to_datetime(ypred['timestamp'])
for i in y_train.columns[1:]:
    ypred[i]=np.nan
    ypred[i]=ypred[i].astype(str)

/Users/juanfelipearias/Library/Python/3.9/lib/python/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (1,3,5,11,20,49,52,54,55,56,64,70,71,94,95,101,102,114,129,131,132,133,138,139,152,157,163,174,180,184,193,196,201,207,208,219,220,228,241,245,250,256,258,264,265,269,270,271,272,273) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


# Utilitary data

In [257]:
s_id=[]
latitude=[]
longitude=[]
for i in y_localisation[['s_id','latitude','longitude']].values.tolist():
    if i[0] not in s_id:
        s_id.append(i[0])
    if i[1] not in latitude:
        latitude.append(i[1])
    if i[2] not in longitude:
        longitude.append(i[2])

#  organizing the data

In [258]:
cols_ordered=["S"+str(i)+"-T"+str(j) for i in range(1,100) for j in range(1,4)]
cols_ordered=list(filter(lambda x: x in y_train.columns.tolist(),cols_ordered))
cols_ordered.insert(0,y_train.columns.tolist()[0])
y_train=y_train[cols_ordered]
y_localisation['closest_s_id']=y_localisation.apply(closest_station,axis=1)

In [222]:
y_localisation.loc[y_localisation.s_id=='S1'].closest_s_id.unique()[0]

'S29'

In [259]:
ypred=ypred.apply(pred,axis=1)

In [261]:
ypred.to_csv('blsubmission.csv')